# Monty Hall Environment

L'agent est un candidat au jeu Monty Hall, Il doit prendre 2 décisions successives. Dans cet 
environnement, il y a 3 portes A, B et C. Au démarrage de l'environnement, une porte est tirée au hasard 
de manière cachée pour l'agent, il s'agit de la porte gagnante. La première action de l'agent est de choisir 
une porte parmi les trois portes. Ensuite, une porte parmi les 2 restantes non choisies par l'agent est 
retirée du jeu, il s'agit forcément d'une porte non gagnante. L'agent ensuite doit effectuer une nouvelle 
action : choisir de conserver la porte choisie au départ ou changer pour la porte restante. Une fois le choix 
fait, la porte choisie est 'ouverte' et l'on découvre si elle était gagnante (reward de 1.0) ou non (reward de 
0.0).


In [30]:
import numpy as np
from typing import List, Tuple
from tqdm import tqdm

# Classe MontyHall Level1

In [31]:
from montyhall import MontyHallEnvLevel2

# Algorithme Naïf Q-learning

L'algorithme naïf de Q-learning est une méthode d'apprentissage par renforcement sans modèle qui cherche à apprendre la politique optimale pour un agent en explorant et en exploitant un environnement. Cet algorithme utilise une table de valeurs Q pour stocker et mettre à jour les estimations des récompenses futures attendues pour chaque paire état-action. Voici les étapes clés de cet algorithme :

1. **Initialisation** : La table Q est initialement remplie de valeurs arbitraires, et ces valeurs sont progressivement ajustées à travers les épisodes d'apprentissage.

2. **Choix des actions** : À chaque étape, l'agent choisit une action basée soit sur l'exploration (choix aléatoire) pour découvrir de nouvelles stratégies, soit sur l'exploitation (choisir l'action avec la valeur Q la plus élevée pour l'état actuel) pour maximiser les récompenses.

3. **Mise à jour de Q** : Après avoir pris une action, l'agent observe la récompense obtenue et le nouvel état atteint. La valeur Q pour la paire état-action est ensuite mise à jour en utilisant la formule :
   \[
   Q(s, a) = Q(s, a) + \alpha \left[r + \gamma \max_{a'} Q(s', a') - Q(s, a)\right]
   \]
   où \( \alpha \) est le taux d'apprentissage, \( \gamma \) est le facteur de dépréciation, \( r \) est la récompense observée, \( s' \) est le nouvel état, et \( a' \) est la meilleure action possible dans le nouvel état.

4. **Répétition** : Ces étapes sont répétées pour de nombreux épisodes jusqu'à ce que la table Q converge vers une approximation de la fonction de valeur optimale, permettant à l'agent de suivre la politique optimale déduite de Q.

L'algorithme de Q-learning est particulièrement utile pour les problèmes avec un espace d'état et d'action discrets et peut être appliqué à une variété de tâches d'apprentissage et de décision.


In [32]:
def naive_q_learning(env_type, alpha: float = 0.1, epsilon: float = 0.1, gamma: float = 0.999, nb_iter: int = 100000):
    Q = {}

    env = env_type()
    for it in tqdm(range(nb_iter)):
        env.reset()

        while not env.is_game_over():
            s = env.state_id()
            aa = env.available_actions()

            if s not in Q:
                Q[s] = {}
                for a in aa:
                    Q[s][a] = np.random.random()  # Initialize Q-values for each state-action pair

            # Decide whether to explore or exploit
            if np.random.random() < epsilon:
                a = np.random.choice(aa)
            else:
                # Choose the best action based on current Q-values
                q_s = [Q[s][a] for a in aa]
                best_a_index = np.argmax(q_s)
                a = aa[best_a_index]

            prev_score = env.score()
            env.step(a)
            r = env.score() - prev_score

            s_p = env.state_id()
            aa_p = env.available_actions()

            if env.is_game_over():
                target = r
            else:
                if s_p not in Q:
                    Q[s_p] = {}
                    for a_p in aa_p:
                        Q[s_p][a_p] = np.random.random()
                q_s_p = [Q[s_p][a_p] for a_p in aa_p]
                max_a_p = np.max(q_s_p)
                target = r + gamma * max_a_p

            Q[s][a] = (1 - alpha) * Q[s][a] + alpha * target

    # Extract policy from Q-values
    Pi = {}
    for s, actions in Q.items():
        best_a = max(actions, key=actions.get)
        Pi[s] = best_a

    return Pi, Q

# Example usage:
env = MontyHallEnvLevel2()
naive_q_learning(env_type=env, nb_iter=10000)

100%|██████████| 10000/10000 [00:00<00:00, 55844.46it/s]


({0: 0}, {0: {0: 0.20017937531886448, 1: 0.04467919678750267}})

# Algorithme Monte Carlo avec départs exploratoires (ES)

L'algorithme de Monte Carlo avec départs exploratoires (ES) est une technique d'apprentissage par renforcement qui permet d'estimer la politique optimale en utilisant l'échantillonnage complet des retours (gains cumulés) de chaque épisode. Contrairement aux méthodes basées sur le temps différé comme Q-learning, les méthodes Monte Carlo ajustent les estimations de la politique uniquement à la fin de chaque épisode. Voici les étapes principales de cet algorithme :

1. **Initialisation** : La politique (Pi) et la table de valeurs Q sont initialisées. Pour chaque état, une action est choisie aléatoirement comme action par défaut. Un dictionnaire de retours est également initialisé pour stocker les retours accumulés pour chaque paire état-action.

2. **Départs exploratoires** : Chaque épisode commence dans un état initial choisi aléatoirement avec une action également choisie aléatoirement. Cela assure que toutes les paires état-action ont une chance d'être explorées suffisamment.

3. **Génération de l'épisode** : L'agent suit la politique actuelle sauf pour le premier choix qui est aléatoire. L'épisode est enregistré sous forme de séquence de triplets (état, action, récompense).

4. **Calcul du retour** : À la fin de l'épisode, le retour pour chaque étape est calculé en remontant depuis la fin de l'épisode jusqu'au début, en utilisant le facteur de dépréciation \( \gamma \). Le retour est le gain cumulé à partir de cette étape jusqu'à la fin de l'épisode.

5. **Mise à jour de Q et de la politique** :
   - **Mise à jour de Q** : Pour chaque paire état-action unique rencontrée dans l'épisode, le retour est ajouté à la liste des retours correspondants et la valeur Q est mise à jour comme la moyenne de ces retours.
   - **Mise à jour de la politique** : Pour chaque état visité, la politique est mise à jour pour choisir l'action qui maximise la valeur Q estimée pour cet état.

6. **Répétition** : Le processus est répété pour un grand nombre d'épisodes pour permettre à la politique de converger vers la politique optimale.

Cet algorithme est particulièrement utile dans les environnements avec des dynamiques simples où la méthode des départs exploratoires est faisable. Il est aussi bien adapté aux situations où il est possible de simuler l'environnement pour générer des épisodes avec des conditions de départ arbitraires.


In [33]:
import numpy as np
from tqdm import tqdm
from collections import defaultdict

def naive_monte_carlo_with_exploring_starts(env_type, gamma=0.999, nb_iter=10000, max_steps=10):
    Pi = {}
    Q = defaultdict(lambda: 0.0)
    Returns = defaultdict(list)

    for it in tqdm(range(nb_iter)):
        env = env_type.from_random_state()
        is_first_action = True
        trajectory = []
        steps_count = 0
        
        while not env.is_game_over() and steps_count < max_steps:
            s = env.state_id()
            aa = env.available_actions()

            if is_first_action:
                a = np.random.choice(aa)
                is_first_action = False
            else:
                if s not in Pi:
                    Pi[s] = np.random.choice(aa)
                a = Pi[s]

            prev_score = env.score()
            env.step(a)
            r = env.score() - prev_score

            trajectory.append((s, a, r, aa))
            steps_count += 1

        G = 0
        for (t, (s, a, r, aa)) in reversed(list(enumerate(trajectory))):
            G = gamma * G + r

            if all(map(lambda triplet: triplet[0] != s or triplet[1] != a, trajectory[:t])):
                Returns[(s, a)].append(G)
                Q[(s, a)] = np.mean(Returns[(s, a)])

                best_a = None
                best_a_score = float('-inf')
                for a in aa:
                    if (s, a) not in Q:
                        Q[(s, a)] = np.random.random()
                    if Q[(s, a)] > best_a_score:
                        best_a = a
                        best_a_score = Q[(s, a)]

                Pi[s] = best_a

    return Pi

# Run the Monte Carlo ES algorithm
optimal_policy_mc = naive_monte_carlo_with_exploring_starts(MontyHallEnvLevel2, nb_iter=10000)
print(optimal_policy_mc)


  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:00<00:00, 70723.34it/s]

{}


# Algorithme On-policy First visit Monte Carlo control

In [34]:
import numpy as np
from tqdm import tqdm
from collections import defaultdict

def on_policy_first_visit_mc_control(env_type, gamma=0.999, epsilon=0.1, nb_iter=10000, max_steps=10):
    Pi = {}
    Q = defaultdict(lambda: 0.0)
    Returns = defaultdict(list)

    for it in tqdm(range(nb_iter)):
        env = env_type.from_random_state()
        trajectory = []
        steps_count = 0

        while not env.is_game_over() and steps_count < max_steps:
            s = env.state_id()
            aa = env.available_actions()

            if np.random.random() < epsilon or s not in Pi:
                a = np.random.choice(aa)
            else:
                a = Pi[s]

            prev_score = env.score()
            env.step(a)
            r = env.score() - prev_score

            trajectory.append((s, a, r, aa))
            steps_count += 1

        G = 0
        visited_state_action_pairs = set()
        for (s, a, r, aa) in reversed(trajectory):
            G = gamma * G + r

            if (s, a) not in visited_state_action_pairs:
                visited_state_action_pairs.add((s, a))
                Returns[(s, a)].append(G)
                Q[(s, a)] = np.mean(Returns[(s, a)])

                best_a = max(aa, key=lambda action: Q[(s, action)])
                Pi[s] = best_a

    return Pi

# Run the On-Policy First Visit MC Control algorithm
optimal_policy_q = on_policy_first_visit_mc_control(MontyHallEnvLevel2, nb_iter=10000)
print(optimal_policy_q)


100%|██████████| 10000/10000 [00:00<00:00, 68350.88it/s]

{}


# Off Policy First visit Monte Carlo Control

In [35]:
def off_policy_mc_control(env_type, gamma=0.999, nb_iter=10000, max_steps=10):
    Q = defaultdict(lambda: 0.0)
    C = defaultdict(lambda: 0.0)
    b = defaultdict(lambda: 1.0)  # behavior policy (random)

    Pi = {}

    for it in tqdm(range(nb_iter)):
        env = env_type.from_random_state()
        trajectory = []
        steps_count = 0

        while not env.is_game_over() and steps_count < max_steps:
            s = env.state_id()
            aa = env.available_actions()

            a = np.random.choice(aa)
            prev_score = env.score()
            env.step(a)
            r = env.score() - prev_score

            trajectory.append((s, a, r))
            steps_count += 1

        G = 0
        W = 1.0
        for (s, a, r) in reversed(trajectory):
            G = gamma * G + r
            C[(s, a)] += W
            Q[(s, a)] += (W / C[(s, a)]) * (G - Q[(s, a)])
            
            Pi[s] = max(env.available_actions(), key=lambda action: Q[(s, action)])
            
            if a != Pi[s]:
                break
            
            W *= 1.0 / b[(s, a)]

    return Pi

off_policy_mc_control(env, nb_iter=10000)


100%|██████████| 10000/10000 [00:00<00:00, 73883.18it/s]


{}

# Sarsa

In [36]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


def build_model(state_size, action_size):
    model = Sequential()
    model.add(Dense(24, input_dim=state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

def sarsa(env_class, num_episodes, alpha, gamma, epsilon):
    env = env_class()
    state_size = env.num_states()
    action_size = env.num_actions()
    model = build_model(state_size, action_size)

    for episode in range(num_episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        done = False
        action = np.random.choice(env.available_actions()) if np.random.rand() < epsilon else np.argmax(model.predict(state)[0])

        while not done:
            next_state, reward, done = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])
            next_action = np.random.choice(env.available_actions()) if np.random.rand() < epsilon else np.argmax(model.predict(next_state)[0])

            target = reward + gamma * model.predict(next_state)[0][next_action] if not done else reward
            target_f = model.predict(state)
            target_f[0][action] = (1 - alpha) * target_f[0][action] + alpha * target

            model.fit(state, target_f, epochs=1, verbose=0)

            state = next_state
            action = next_action

    return model

# Parameters for SARSA
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000  # Number of episodes

# Create the environment and run the SARSA algorithm
model = sarsa(MontyHallEnvLevel2, num_episodes, alpha, gamma, epsilon)

# Display the optimal policy found
env = MontyHallEnvLevel2()
optimal_policy = []

for state in range(env.num_states()):
    state_input = np.reshape(state, [1, env.num_states()])
    optimal_action = np.argmax(model.predict(state_input)[0])
    optimal_policy.append(optimal_action)
    print(f"State {state}: Optimal Action {optimal_action}")

ValueError: cannot reshape array of size 1 into shape (1,25)

# Dyna Q

In [ ]:
import random
import numpy as np

def epsilon_greedy_policy(Q, state, epsilon, available_actions):
    if random.uniform(0, 1) < epsilon:
        return random.choice(available_actions)
    else:
        return max(available_actions, key=lambda x: Q[state][x])

def dyna_q(env, num_episodes, alpha, gamma, epsilon, planning_steps):
    Q = {state: [0.0, 0.0] for state in range(env.num_states())}  # Only 2 actions: stick (0) or switch (1)
    model = {state: {} for state in range(env.num_states())}

    for episode in range(num_episodes):
        env.reset()
        state = env.state_id()
        available_actions = env.available_actions()
        action = epsilon_greedy_policy(Q, state, epsilon, available_actions)

        while not env.is_game_over():
            next_state, reward, done = env.step(action)
            next_available_actions = env.available_actions()
            next_action = epsilon_greedy_policy(Q, next_state, epsilon, next_available_actions)

            # Update Q
            Q[state][action] += alpha * (reward + gamma * Q[next_state][next_action] - Q[state][action])

            # Update the model
            model[state][action] = (reward, next_state)

            # Planning
            for _ in range(planning_steps):
                sampled_state = random.choice(list(model.keys()))
                if not model[sampled_state]:
                    continue
                sampled_action = random.choice(list(model[sampled_state].keys()))
                sampled_reward, sampled_next_state = model[sampled_state][sampled_action]

                Q[sampled_state][sampled_action] += alpha * (sampled_reward + gamma * max(Q[sampled_next_state]) - Q[sampled_state][sampled_action])

            state = next_state
            action = next_action

    return Q

# Parameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Exploration rate
num_episodes = 1000  # Number of episodes
planning_steps = 5  # Number of planning steps

# Create the environment and run the Dyna-Q algorithm
Q = dyna_q(env, num_episodes, alpha, gamma, epsilon, planning_steps)

# Display the optimal policy found
for state in range(env.num_states()):
    best_action = np.argmax(Q[state])
    action_symbols = ['Stick', 'Switch']
    print(f"State {state}: {action_symbols[best_action]}")


KeyError: 47

# Policy et Value Iteration

In [ ]:
class PolicyIteration:
    def __init__(self, env_class, gamma=0.99, theta=1e-3):
        self.env_class = env_class
        self.gamma = gamma
        self.theta = theta
        self.env = env_class()
        self.num_states = self.env.num_states()
        self.num_actions = self.env.num_actions()

    def run(self) -> Tuple[List[int], List[float]]:
        policy = np.random.choice(self.num_actions, size=self.num_states)
        V = np.zeros(self.num_states)

        while True:
            V = self.policy_evaluation(policy, V)
            policy_stable = self.policy_improvement(policy, V)
            if policy_stable:
                break

        return policy, V

    def policy_evaluation(self, policy, V):
        while True:
            delta = 0
            for s in range(self.num_states):
                v = V[s]
                new_v = 0
                for a in range(self.num_actions):
                    if policy[s] == a:
                        for next_state in range(self.num_states):
                            for r_index in range(self.env.num_rewards()):
                                prob = self.env.p(s, a, next_state, r_index)
                                reward = self.env.reward(r_index)
                                new_v += prob * (reward + self.gamma * V[next_state])
                delta = max(delta, abs(v - new_v))
                V[s] = new_v
            if delta < self.theta:
                break
        return V

    def policy_improvement(self, policy, V):
        policy_stable = True
        for s in range(self.num_states):
            chosen_a = policy[s]
            action_values = self.one_step_lookahead(s, V)
            best_a = np.argmax(action_values)
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = best_a
        return policy_stable

    def one_step_lookahead(self, state, V):
        A = np.zeros(self.num_actions)
        for a in range(self.num_actions):
            for next_state in range(self.num_states):
                for r_index in range(self.env.num_rewards()):
                    prob = self.env.p(state, a, next_state, r_index)
                    reward = self.env.reward(r_index)
                    A[a] += prob * (reward + self.gamma * V[next_state])
        return A


class ValueIteration:
    def __init__(self, env_class, gamma=0.99, theta=1e-3):
        self.env_class = env_class
        self.gamma = gamma
        self.theta = theta
        self.env = env_class()
        self.num_states = self.env.num_states()
        self.num_actions = self.env.num_actions()

    def run(self) -> List[float]:
        V = np.zeros(self.num_states)
        while True:
            delta = 0
            for s in range(self.num_states):
                v = V[s]
                action_values = self.one_step_lookahead(s, V)
                V[s] = max(action_values)
                delta = max(delta, abs(v - V[s]))
            if delta < self.theta:
                break
        return V

    def one_step_lookahead(self, state, V):
        A = np.zeros(self.num_actions)
        for a in range(self.num_actions):
            for next_state in range(self.num_states):
                for r_index in range(self.env.num_rewards()):
                    prob = self.env.p(state, a, next_state, r_index)
                    reward = self.env.reward(r_index)
                    A[a] += prob * (reward + self.gamma * V[next_state])
        return A


In [ ]:
# Create the environment
env_class = MontyHallEnvLevel2()

# Run Policy Iteration
pi = PolicyIteration(env_class)
policy, V = pi.run()

# Display the optimal policy found
action_symbols = ['Stick', 'Switch']
for state in range(env_class().num_states()):
    print(f"State {state}: {action_symbols[policy[state]]}")
print(V)


State 0: Stick
State 1: Stick
State 2: Stick
State 3: Stick
State 4: Stick
State 5: Stick
State 6: Stick
State 7: Stick
State 8: Stick
State 9: Stick
State 10: Stick
State 11: Stick
State 12: Stick
State 13: Stick
State 14: Stick
State 15: Stick
State 16: Stick
State 17: Stick
State 18: Stick
State 19: Stick
State 20: Stick
State 21: Stick
State 22: Stick
State 23: Stick
State 24: Stick
State 25: Stick
State 26: Stick
State 27: Stick
State 28: Stick
State 29: Stick
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


In [ ]:
# Create the environment


# Run Value Iteration
vi = ValueIteration(env_class)
V = vi.run()

# Display the value function found
for state in range(env_class().num_states()):
    print(f"State {state}: Value {V[state]}")


State 0: Value 0.0
State 1: Value 0.0
State 2: Value 0.0
State 3: Value 0.0
State 4: Value 0.0
State 5: Value 0.0
State 6: Value 0.0
State 7: Value 0.0
State 8: Value 0.0
State 9: Value 0.0
State 10: Value 0.0
State 11: Value 0.0
State 12: Value 0.0
State 13: Value 0.0
State 14: Value 0.0
State 15: Value 0.0
State 16: Value 0.0
State 17: Value 0.0
State 18: Value 0.0
State 19: Value 0.0
State 20: Value 0.0
State 21: Value 0.0
State 22: Value 0.0
State 23: Value 0.0
State 24: Value 0.0
State 25: Value 0.0
State 26: Value 0.0
State 27: Value 0.0
State 28: Value 0.0
State 29: Value 0.0
